#### 시험장 나누기
---
https://school.programmers.co.kr/learn/courses/30/lessons/81305

In [3]:
class Node:
    def __init__(self, trafic):
        self.trafic = trafic
        self.left = -1
        self.right = -1
    def show(self):
        print(self.trafic, f' - [{self.left}, {self.right}]')

class Tree:
    def __init__(self, num, links):
        self.Nodes = []
        for i in range(len(num)):
            self.Nodes.append(Node(num[i]))
            left, right = links[i]
            if left != -1:
                self.Nodes[-1].left = left
            if right != -1:
                self.Nodes[-1].right = right

    def show(self):
        for node in self.Nodes:
            node.show()

        

def solution(k, num, links):
    tree = Tree(num, links)
    tree.show()
    answer = 0
    return answer

In [4]:
solution(3, [12, 30, 1, 8, 8, 6, 20, 7, 5, 10, 4, 1], [[-1, -1], [-1, -1], [-1, -1], [-1, -1], [8, 5], [2, 10], [3, 0], [6, 1], [11, -1], [7, 4], [-1, -1], [-1, -1]])

12  - [-1, -1]
30  - [-1, -1]
1  - [-1, -1]
8  - [-1, -1]
8  - [8, 5]
6  - [2, 10]
20  - [3, 0]
7  - [6, 1]
5  - [11, -1]
10  - [7, 4]
4  - [-1, -1]
1  - [-1, -1]


0